In [9]:
# This Code cell removes all files in the directory in the case I make a mistake and need to start over from the beginning

import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the root directory for processing
ROOT_DIRECTORY = r'C:\Users\helix\Desktop\CFA Receiving Report 2024'

def delete_excel_files_in_directory(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(directory, filename)
            try:
                os.remove(file_path)
                logger.info(f'Successfully deleted {file_path}')
            except Exception as e:
                logger.error(f'Error deleting {file_path}: {e}')

def delete_excel_files_in_all_months(root_directory):
    for month_name in os.listdir(root_directory):
        month_directory = os.path.join(root_directory, month_name)
        output_directory = os.path.join(month_directory, 'Output')
        if os.path.isdir(output_directory):
            logger.info(f'Processing output directory: {output_directory}')
            delete_excel_files_in_directory(output_directory)

if __name__ == "__main__":
    delete_excel_files_in_all_months(ROOT_DIRECTORY)


INFO:__main__:Processing output directory: C:\Users\helix\Desktop\CFA Receiving Report 2024\Apr\Output
INFO:__main__:Successfully deleted C:\Users\helix\Desktop\CFA Receiving Report 2024\Apr\Output\8026291.xlsx
INFO:__main__:Successfully deleted C:\Users\helix\Desktop\CFA Receiving Report 2024\Apr\Output\8030392.xlsx
INFO:__main__:Successfully deleted C:\Users\helix\Desktop\CFA Receiving Report 2024\Apr\Output\8035164.xlsx
INFO:__main__:Successfully deleted C:\Users\helix\Desktop\CFA Receiving Report 2024\Apr\Output\8039419.xlsx
INFO:__main__:Successfully deleted C:\Users\helix\Desktop\CFA Receiving Report 2024\Apr\Output\8043921.xlsx
INFO:__main__:Successfully deleted C:\Users\helix\Desktop\CFA Receiving Report 2024\Apr\Output\8047605.xlsx
INFO:__main__:Successfully deleted C:\Users\helix\Desktop\CFA Receiving Report 2024\Apr\Output\8052712.xlsx
INFO:__main__:Successfully deleted C:\Users\helix\Desktop\CFA Receiving Report 2024\Apr\Output\8056693.xlsx
INFO:__main__:Successfully delete

In [10]:

import os
import re
import pandas as pd
import pdfplumber
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

ROOT_DIRECTORY = r'C:\Users\helix\Desktop\CFA Receiving Report 2024'

COLUMNS_TO_KEEP = ["Supplier Code", "Description", "Unit Cost", "Qty Received", "Total Cost"]
SUPPLIER_CODES = [
    '100787', '108173', '108172', '107506', '107507', '043523', '103043', '043524', 
    '077883', '063760', '100090', '100089', '100091', '030160', '107731', '107108', 
    '107109', '032877', '032883', '101402', '107511', '108188', '100798', '100703', 
    '106931', '052481', '101464', '032882', '101463', '032690', '100996', '101625', 
    '016749', '103071', '007872', '100976', '100446', '108174', '100783', '100782', 
    '100781', '063767', '100803', '101595', '100786', '100785', '100970', '100051', 
    '101204', '108176', '107501', '101619', '107736', '100780', '075781', '101024', 
    '101450', '025258', '108127', '107625', '108047'
]


def extract_7_digit_number(base_filename):
    match = re.search(r'_(\d{7})-', base_filename)
    if match:
        return match.group(1)
    else:
        logger.warning(f'No 7-digit number found in filename: {base_filename}')
    return None

def remove_leading_digits(description):
    """Remove leading 5-digit numbers from description strings."""
    cleaned_description = re.sub(r'^\d{5}\s*', '', description)
    return cleaned_description.strip()

def extract_data_from_table(page):
    """Extracts data from a PDF page using table extraction with manual headers."""
    rows = []
    table = page.extract_table()

    if not table:
        logger.warning("No table data extracted from page.")
        return rows

    for row in table:
        if not row or len(row) < 5:  # Adjust based on expected columns in your table
            continue
        
        try:
            supplier_code = row[2].strip() if row[2] else ""  
            description = remove_leading_digits(row[1]).strip() if row[1] else ""
            unit_cost = convert_to_float(row[8]) if row[8] else None
            qty_received = convert_to_float(row[7]) if row[7] else None
            total_cost = convert_to_float(row[9]) if row[9] else None

            # Log warnings if essential data is missing
            if not description or not supplier_code or qty_received is None or unit_cost is None or total_cost is None:
                logger.warning(f"Missing data in row: {row}")
                continue

            # Only keep rows with relevant supplier codes
            if supplier_code in SUPPLIER_CODES:
                row_data = {
                    "Supplier Code": supplier_code,
                    "Description": description,
                    "Qty Received": qty_received,
                    "Unit Cost": unit_cost,
                    "Total Cost": total_cost,
                }
                rows.append(row_data)
                logger.info(f"Extracted data for Supplier Code {supplier_code}: {row_data}")
        except Exception as e:
            logger.warning(f"Error processing row: {row} - {e}")
    return rows

def convert_to_float(value):
    """Convert a string value to float, handling common issues."""
    try:
        return float(str(value).replace('$', '').replace(',', '').strip())
    except (ValueError, TypeError):
        return None  # Return None if the value can't be converted to a float

def process_files_for_month(month_directory):
    """Process all PDF files in the given month's directory."""
    input_directory = os.path.join(month_directory, 'Input')
    output_directory = os.path.join(month_directory, 'Output')

    os.makedirs(output_directory, exist_ok=True)

    for filename in os.listdir(input_directory):
        if filename.lower().endswith('.pdf'):
            file_path = os.path.join(input_directory, filename)
            base_filename = os.path.splitext(filename)[0]
            logger.info(f'Processing file: {base_filename}')

            # Extract the 7-digit number from the filename
            file_number = extract_7_digit_number(base_filename)
            if not file_number:
                continue  # Skip files without a valid 7-digit number

            # Save the Excel file with the extracted 7-digit number as the filename
            output_file_path = os.path.join(output_directory, f'{file_number}.xlsx')

            all_rows = []
            with pdfplumber.open(file_path) as pdf:
                for page_number, page in enumerate(pdf.pages, start=1):
                    logger.info(f"Extracting data from page {page_number} of {filename}")
                    page_rows = extract_data_from_table(page)
                    all_rows.extend(page_rows)

            if all_rows:
                df = pd.DataFrame(all_rows, columns=COLUMNS_TO_KEEP)
                
                # Save the data to an Excel file
                df.to_excel(output_file_path, index=False)
                logger.info(f'Successfully saved to {output_file_path}')
            else:
                logger.warning(f"No data extracted from {filename}")

def process_all_months(root_directory):
    """Process all months by iterating through directories and processing PDF files."""
    for month_name in os.listdir(root_directory):
        month_directory = os.path.join(root_directory, month_name)
        if os.path.isdir(month_directory):
            logger.info(f'Processing month directory: {month_directory}')
            process_files_for_month(month_directory)

if __name__ == "__main__":
    process_all_months(ROOT_DIRECTORY)


INFO:__main__:Processing month directory: C:\Users\helix\Desktop\CFA Receiving Report 2024\Apr
INFO:__main__:Processing file: ReceivingReport_04050_8026291-river.pengelly1
INFO:__main__:Extracting data from page 1 of ReceivingReport_04050_8026291-river.pengelly1.pdf
INFO:__main__:Extracted data for Supplier Code 100090: {'Supplier Code': '100090', 'Description': 'SANITIZER, SOLIDSENSE KAY', 'Qty Received': 1.0, 'Unit Cost': 48.92, 'Total Cost': 48.92}
INFO:__main__:Extracting data from page 2 of ReceivingReport_04050_8026291-river.pengelly1.pdf
INFO:__main__:Extracted data for Supplier Code 101402: {'Supplier Code': '101402', 'Description': 'WIPES, SURFACE SANITIZER', 'Qty Received': 1.0, 'Unit Cost': 37.46, 'Total Cost': 37.46}
INFO:__main__:Extracted data for Supplier Code 107511: {'Supplier Code': '107511', 'Description': 'TOWEL, SANITIZER COMATIBLE BLUE 10.75 X\n21', 'Qty Received': 1.0, 'Unit Cost': 22.53, 'Total Cost': 22.53}
INFO:__main__:Extracted data for Supplier Code 025258:

In [ ]:
import os
import pandas as pd

# Define the root directory and output file for the monthly aggregation
ROOT_DIRECTORY = r'C:\Users\helix\Desktop\CFA Receiving Report 2024'
COLUMNS_TO_KEEP = ["Supplier Code", "Description", "Unit Cost", "Qty Received", "Total Cost"]
OUTPUT_FILE = 'Monthly_Truck_Order_Summary.xlsx'

def convert_to_float(value):
    """Convert currency string to float."""
    try:
        return float(str(value).replace('$', '').replace(',', ''))
    except (ValueError, TypeError):
        return 0.0

def clean_qty_received(value):
    """Ensure the 'Qty Received' is converted to a consistent numeric format."""
    try:
        return int(float(str(value).replace(',', '').strip()))
    except (ValueError, TypeError):
        return 0  # Return 0 if conversion fails

def format_as_currency(value):
    """Format a numeric value as currency."""
    try:
        return f"${value:,.2f}"
    except (ValueError, TypeError):
        return value

def aggregate_monthly_data(month_directory):
    """Aggregate all Excel files in the 'Output' directory into a single monthly summary."""
    output_directory = os.path.join(month_directory, 'Output')
    aggregated_data = pd.DataFrame(columns=COLUMNS_TO_KEEP)

    for filename in os.listdir(output_directory):
        if filename.lower().endswith('.xlsx'):
            file_path = os.path.join(output_directory, filename)
            try:
                df = pd.read_excel(file_path)
                
                # Convert the Total Cost and Unit Cost columns to numeric values
                df['Total Cost'] = df['Total Cost'].apply(convert_to_float)
                df['Unit Cost'] = df['Unit Cost'].apply(convert_to_float)
                
                # Clean 'Qty Received' column to ensure it's numeric and consistent
                df['Qty Received'] = df['Qty Received'].apply(clean_qty_received)

                aggregated_data = pd.concat([aggregated_data, df], ignore_index=True)
            except Exception as e:
                print(f"Error reading file {file_path}: {e}")

    # Aggregate the data by summing quantities and total costs for each unique item
    aggregated_summary = aggregated_data.groupby(["Supplier Code","Description", "Unit Cost"], as_index=False).agg({
        "Qty Received": "sum",
        "Total Cost": "sum"
    })

    # Calculate the total of all "Total Cost" entries
    total_cost_sum = aggregated_summary['Total Cost'].sum()

    # Append the total row at the end of the DataFrame
    total_row = pd.DataFrame({
        "Supplier Code": ["Total Monthly Cost"],
        "Description": [""],
        "Unit Cost": [""],
        "Qty Received": [""],
        "Total Cost": [total_cost_sum]
    })

    aggregated_summary = pd.concat([aggregated_summary, total_row], ignore_index=True)

    # Convert numeric columns back to currency format
    aggregated_summary['Total Cost'] = aggregated_summary['Total Cost'].apply(format_as_currency)
    aggregated_summary['Unit Cost'] = aggregated_summary['Unit Cost'].apply(format_as_currency)

    # Save the aggregated data
    summary_file_path = os.path.join(output_directory, OUTPUT_FILE)
    aggregated_summary.to_excel(summary_file_path, index=False)
    print(f'Successfully saved monthly summary to {summary_file_path}')

def process_all_months(root_directory):
    """Process all months by iterating through all directories and aggregating the data."""
    for month_name in os.listdir(root_directory):
        month_directory = os.path.join(root_directory, month_name)
        if os.path.isdir(month_directory):
            print(f'Processing month directory: {month_directory}')
            aggregate_monthly_data(month_directory)

# Execute the aggregation process for all months
process_all_months(ROOT_DIRECTORY)


C:\Users\helix\AppData\Local\Temp\ipykernel_11632\1691575617.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aggregated_data = pd.concat([aggregated_data, df], ignore_index=True)


Processing month directory: C:\Users\helix\Desktop\CFA Receiving Report 2024\Apr
Successfully saved monthly summary to C:\Users\helix\Desktop\CFA Receiving Report 2024\Apr\Output\Monthly_Truck_Order_Summary.xlsx
Processing month directory: C:\Users\helix\Desktop\CFA Receiving Report 2024\Aug


C:\Users\helix\AppData\Local\Temp\ipykernel_11632\1691575617.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aggregated_data = pd.concat([aggregated_data, df], ignore_index=True)


Successfully saved monthly summary to C:\Users\helix\Desktop\CFA Receiving Report 2024\Aug\Output\Monthly_Truck_Order_Summary.xlsx
Processing month directory: C:\Users\helix\Desktop\CFA Receiving Report 2024\Dec
Successfully saved monthly summary to C:\Users\helix\Desktop\CFA Receiving Report 2024\Dec\Output\Monthly_Truck_Order_Summary.xlsx
Processing month directory: C:\Users\helix\Desktop\CFA Receiving Report 2024\Feb


C:\Users\helix\AppData\Local\Temp\ipykernel_11632\1691575617.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aggregated_data = pd.concat([aggregated_data, df], ignore_index=True)


...

Successfully saved monthly summary to C:\Users\helix\Desktop\CFA Receiving Report 2024\Nov\Output\Monthly_Truck_Order_Summary.xlsx
Processing month directory: C:\Users\helix\Desktop\CFA Receiving Report 2024\Oct


C:\Users\helix\AppData\Local\Temp\ipykernel_11632\1691575617.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aggregated_data = pd.concat([aggregated_data, df], ignore_index=True)


Successfully saved monthly summary to C:\Users\helix\Desktop\CFA Receiving Report 2024\Oct\Output\Monthly_Truck_Order_Summary.xlsx
Processing month directory: C:\Users\helix\Desktop\CFA Receiving Report 2024\Sep


C:\Users\helix\AppData\Local\Temp\ipykernel_11632\1691575617.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aggregated_data = pd.concat([aggregated_data, df], ignore_index=True)


Successfully saved monthly summary to C:\Users\helix\Desktop\CFA Receiving Report 2024\Sep\Output\Monthly_Truck_Order_Summary.xlsx


In [12]:
import shutil
import os

# Define the root directory and the new directory for storing all monthly summaries
ROOT_DIRECTORY = r'C:\Users\helix\Desktop\CFA Receiving Report 2024'
SUMMARY_DIRECTORY = os.path.join(ROOT_DIRECTORY, 'Monthly Summaries')

def collect_monthly_summaries(root_directory, summary_directory):
    """Collect all monthly summary Excel files and move them to a new directory."""
    
    # Create the new directory if it doesn't exist
    if not os.path.exists(summary_directory):
        os.makedirs(summary_directory)
        print(f'Created directory: {summary_directory}')

    # Iterate through each month's directory to find the monthly summary file
    for month_name in os.listdir(root_directory):
        month_directory = os.path.join(root_directory, month_name, 'Output')
        
        if os.path.isdir(month_directory):
            summary_file_path = os.path.join(month_directory, 'Monthly_Truck_Order_Summary.xlsx')
            
            if os.path.exists(summary_file_path):
                # Move the summary file to the new directory
                destination_path = os.path.join(summary_directory, f'{month_name}_Summary.xlsx')
                shutil.copy(summary_file_path, destination_path)
                print(f'Moved {summary_file_path} to {destination_path}')
            else:
                print(f'No summary file found in: {month_directory}')

if __name__ == "__main__":
    collect_monthly_summaries(ROOT_DIRECTORY, SUMMARY_DIRECTORY)


Created directory: C:\Users\helix\Desktop\CFA Receiving Report 2024\Monthly Summaries
Moved C:\Users\helix\Desktop\CFA Receiving Report 2024\Apr\Output\Monthly_Truck_Order_Summary.xlsx to C:\Users\helix\Desktop\CFA Receiving Report 2024\Monthly Summaries\Apr_Summary.xlsx
Moved C:\Users\helix\Desktop\CFA Receiving Report 2024\Aug\Output\Monthly_Truck_Order_Summary.xlsx to C:\Users\helix\Desktop\CFA Receiving Report 2024\Monthly Summaries\Aug_Summary.xlsx
Moved C:\Users\helix\Desktop\CFA Receiving Report 2024\Dec\Output\Monthly_Truck_Order_Summary.xlsx to C:\Users\helix\Desktop\CFA Receiving Report 2024\Monthly Summaries\Dec_Summary.xlsx
Moved C:\Users\helix\Desktop\CFA Receiving Report 2024\Feb\Output\Monthly_Truck_Order_Summary.xlsx to C:\Users\helix\Desktop\CFA Receiving Report 2024\Monthly Summaries\Feb_Summary.xlsx
Moved C:\Users\helix\Desktop\CFA Receiving Report 2024\Jan\Output\Monthly_Truck_Order_Summary.xlsx to C:\Users\helix\Desktop\CFA Receiving Report 2024\Monthly Summaries\J